In [25]:
import numpy as np
import pandas as pd
import tqdm

In [26]:
# Load the dataset

def read_csv(file_path):
    try:
        df = pd.read_csv(file_path, low_memory=False)
        return df
    except Exception:
        return pd.read_csv(file_path, delimiter='_', low_memory=False)
    
def print_to_csv(file_path, results):
    with open(file_path, 'w', encoding='utf-8') as f:
        for res in results:
            f.write(res + '\n')
    f.close()

tables = []
for i in range(0, 19):
    ct = read_csv(f'lake33/table_{i}.csv')
    ct.name = f'table_{i}.csv'
    tables.append(ct)
    
tables_clean = []
for i in range(0, 19):
    if i in [6,7,10]:
        continue
    ct = read_csv(f'lake33c/cleaned_table_{i}.csv')
    ct.name = f'cleaned_table_{i}.csv'
    tables_clean.append(ct)

This notebook implements three methods to find similar columns across different tables in a dataset. The methods are:
1. Set Containment method: Similarity between column values of different tables. \
2. Column name method: Similarity between column names of different tables. Measured in two different ways: 
    - Levenshtein distance (Shows how many single-character edits are needed to change one string into another) 
    - Jaccard similarity on shingles.
3. JOSIE method: Similarity between column values of different tables, using JOSIE. Creates posting lists and prints a top k list of similar columns (k=3).

Thresholds have been set to 0.8, to avoid too many false positives.

In [27]:
def set_containment(set_a, set_b):
    """ Returns the containment of setA in setB """
    if len(set_a) == 0:
        return 0
    return len(set_a.intersection(set_b)) / len(set_a)

In [28]:
# Set Containment method:
def set_containment_method(dataset):
    threshold = 0.8
    
    rows_list = []
    # Iterate through pairs of tables
    for i, df1 in enumerate(tqdm.tqdm(dataset)):
        for j, df2 in enumerate(dataset):
            if i >= j:
                continue
            # Iterate through pairs of columns of both tables:
            for colidx1, col1 in enumerate(df1.columns):
                for colidx2, col2 in enumerate(df2.columns):
                    vals1 = set(df1[col1].dropna())
                    vals2 = set(df2[col2].dropna())
                    
                    sc1 = set_containment(vals1, vals2)
                    if sc1 >= threshold:
                        dict = {}
                        dict.update({'Dataset 1': df1.name, 'Column 1': str(col1), 'Dataset 2': df2.name, 'Column 2': str(col2), 'Set Containment': sc1, 'Relation': 'JOIN'})
                        rows_list.append(dict)
                    
    result = pd.DataFrame(rows_list, columns=['Dataset 1', 'Dataset 2', 'Relation', 'Column 1', 'Column 2', 'Set Containment'])
    return result

result_sc_unclean = set_containment_method(tables)
result_sc_unclean.to_csv('outputs/set_containment_results_unclean.csv', encoding='utf-8', index=False, header=True)

result_sc_clean = set_containment_method(tables_clean)
result_sc_clean.to_csv('outputs/set_containment_results_clean.csv', encoding='utf-8', index=False, header=True)

100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [07:09<00:00, 26.82s/it]


Results method 1: (Runtime ~ 10 minutes)

We notice the set containment method manages to identify many similarities between tables. For instance, it correctly captures similarities between tables 1 and 2, which appear to have similar data, even when the column titles are mismatched. This is the case for many tables, where the column names are different but the values are similar. 

However, this method yields a large number of false positives as well. For example, some tables appear to contain some sort of sensor data, which may have a small range of values that can potentially overlap with some other columns that have no connection to sensors, but a wide range of values. This yields a set containment value that exceeds the threshold, but the columns are not actually similar in any meaningful way.

One method to mitigate this issue would be to consider the set containment relation in both directions, however that would significantly increase the computation time.

In [29]:
# Column name method
# Two ideas: Levenshtein distance, or Jaccard similarity on shingles.
def levenshtein(s1, s2):
    m = len(s1)
    n = len(s2)
    
    prev_row = [i for i in range(n + 1)]
    curr_row = [0] * (n + 1)
    
    for i in range(1, m + 1):
        curr_row[0] = i
        for j in range(1, n + 1):
            if s1[i - 1] == s2[j - 1]:
                curr_row[j] = prev_row[j - 1]
            else:
                curr_row[j] = min(prev_row[j - 1], prev_row[j], curr_row[j - 1]) + 1
        prev_row = curr_row.copy()
    return curr_row[n]

def jaccard_similarity(s1, s2, k=2):
    def get_shingles(s, k):
        return {s[i:i+k] for i in range(len(s) - k + 1)}
    
    shingles1 = get_shingles(s1, k)
    shingles2 = get_shingles(s2, k)
    
    intersection = len(shingles1.intersection(shingles2))
    union = len(shingles1.union(shingles2))
    
    if union == 0:
        return 0.0
    return intersection / union

def column_names(dataset):
    threshold = 0.8
    rows_list = []
    for i, df1 in enumerate(tqdm.tqdm(dataset)):
        for j, df2 in enumerate(dataset):
            if i >= j:
                continue
            for colidx1, col1 in enumerate(df1.columns):
                for colidx2, col2 in enumerate(df2.columns):
                    c1 = col1.lower()
                    c2 = col2.lower()
                    has_result = False
                    
                    dict = {'Dataset 1': df1.name, 'Column 1': str(col1), 'Dataset 2': df2.name, 'Column 2': str(col2), 'Relation': 'UNION'}
                    
                    lev_ratio = (levenshtein(col1.lower(), col2.lower())) / max(len(c1), len(c2))
                    if lev_ratio <= (1 - threshold):
                        has_result = True
                        dict.update({'Levenshtein Ratio': 1 - lev_ratio})
                    
                    for k in range(2,6):
                        sim = jaccard_similarity(c1, c2, k)
                        if sim >= threshold:
                            has_result = True
                            dict.update({f'Jaccard (k={k})': sim})
                    if has_result:
                        rows_list.append(dict)
                  
    result = pd.DataFrame(rows_list, columns=['Dataset 1', 'Dataset 2', 'Relation', 'Column 1', 'Column 2', 'Levenshtein Ratio', 'Jaccard (k=2)', 'Jaccard (k=3)', 'Jaccard (k=4)', 'Jaccard (k=5)'])
    return result

result_cn_unclean = column_names(tables)
result_cn_unclean.to_csv('outputs/column_name_results_unclean.csv', encoding='utf-8', index=False, header=True)

result_cn_clean = column_names(tables_clean)
result_cn_clean.to_csv('outputs/column_name_results_clean.csv', encoding='utf-8', index=False, header=True)

100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:11<00:00,  1.41it/s]


Results methods 2: (Runtime ~ 75 seconds)

The column name method is faster, as we only analyse the column names instead of searching through the whole values. The Levenshtein distance method manages to capture similarities when the column names are very similar. The threshold has been set to 20% difference, which means that the strings can differ by at most 20% of the length of the longer string. This may exclude shorter strings that are similar, but works well for longer strings which may contain spelling mistakes or shortcuts.

The Jaccard similarity method on shingles captures similarities when column names have similar substrings. This works well for column names that differ by many characters semantically, but share common words or abbreviations.

On the other hand, if the names are completely different, both methods cannot capture any similarity between tables, leading to a large number of false negatives.

In [30]:
# JOSIE method

def josie(dataset):
    # Create posting lists
    postings = {}
    for i, df in enumerate(tqdm.tqdm(dataset)):
        for j, col in enumerate(df.columns):
            vals = set(df[col].dropna())
            for entry in vals:
                entry = str(entry)
                if entry not in postings:
                    postings[entry] = set()
                postings[entry].add((df.name, j))
                
    k = 3
    rows_list = []
    
    for i, df1 in enumerate(tqdm.tqdm(dataset)):
        for j, col1 in enumerate(df1.columns):
            vals1 = set(df1[col1].dropna())
            counter = {}
            for val in vals1:
                val = str(val)
                if val in postings:
                    for (table_id, col_id) in postings[val]:
                        if table_id == df1.name: # Do not match within the same table
                            continue
                        if (table_id, col_id) not in counter:
                            counter[(table_id, col_id)] = 0
                        counter[(table_id, col_id)] += 1
            sorted_counter = sorted(counter.items(), key=lambda x: x[1], reverse=True)
            has_result = False
            dict = {'Dataset 1': df1.name, 'Column 1': str(col1)}
            for ((table_id, col_id), count) in sorted_counter[:(min(len(counter), k))]:
                has_result = True
                dict.update({f'Similar Column {len(dict)-1}': f'Table {table_id}, Column {col_id}, Overlap: {count}'})
            if has_result:
                rows_list.append(dict)
                
    col_names = ['Dataset 1', 'Column 1']
    for i in range(k):
        col_names.append(f'Similar Column {i+1}')
    
    result = pd.DataFrame(rows_list, columns=col_names)
    return result

result_josie_unclean = josie(tables)
result_josie_unclean.to_csv('outputs/josie_results_unclean.csv', encoding='utf-8', index=False, header=True)

result_josie_clean = josie(tables_clean)
result_josie_clean.to_csv('outputs/josie_results_clean.csv', encoding='utf-8', index=False, header=True)

100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:04<00:00,  3.22it/s]


Results method 3: (Runtime ~ 15 seconds)

JOSIE does great at creating posting lists quickly, and uses them to quickly find similar columns. It is very fast and also creates a top k list of similar columns, which is useful for potential matches.

However, this method only uses overlap as a metric, which is not giving very insightful information on its own. This method works best if combined with other previous approaches, which could give a better idea if two columns are actually similar or not.